In [10]:
import numpy as np
from src.utils import *

In [17]:
def tournament_selection(fitness, n):
    """
    Perform tournament selection
    
    Args:
        fitness (array-like): Fitness values of the population.
        n (int): Subset size of winners and losers.
        
    Returns:
        best_n (list): Indices of best n individuals.
        worst_n (list): Indices of worst n individuals
    """
    sorted_indices = np.argsort(fitness)
    best_n = sorted_indices[:n]
    worst_n = sorted_indices[-n:][::-1]
    return best_n, worst_n

In [ ]:
fit = np.arange(20)
fit

In [ ]:
best_n, worst_n = tournament_selection(fit, 3)
print(best_n)
print(worst_n)

In [83]:
import numpy as np
import random
import math
operators=[np.add, np.subtract, np.multiply, np.sin, np.cos, np.exp]
unary_operators=[np.sin, np.cos, np.exp]

def is_operator(val):
        return val in operators

class Node:
    op_list = {
        np.add: '+',
        np.subtract: '-',
        np.multiply: '*',
        np.sin: 'sin',
        np.cos: 'cos',
        np.exp: 'exp',  
    }
    def __init__(self, value=None, feature_index=None, left=None, right=None):
        self.value = value 
        self.feature_index = feature_index
        self.left = left
        self.right = right

    def evaluate(self,x=None):
        # first check if it is an operator 
        # if not proceed
        
        if not is_operator(self.value):
            #print("index",self.feature_index)
            if self.feature_index!= None:
                #print("ben inputun indexiyim")
                return x[self.feature_index]
            else:
                #print("ben bir sayıyım")
                return self.value
        # if it is an operator
        if self.value in unary_operators:
            operand_value = self.left.evaluate(x)
            #print("tek",operand_value)
            return self.value(operand_value)
        
        left_value = self.left.evaluate(x)
        right_value = self.right.evaluate(x)
        #print(left_value)
        #print(right_value)
        return self.value(left_value, right_value)
    def __str__(self):
        if not is_operator(self.value):
            if self.feature_index != None:
                return f"x[{self.feature_index}]"
            return str(self.value)

        operator_symbol = self.op_list[self.value]

        if self.value in {np.sin, np.cos, np.log, np.exp}:
            return f"{operator_symbol}({self.left})"

        return f"({self.left} {operator_symbol} {self.right})"

def random_tree(depth, num_features):
    if depth == 0:
        if random.random() < 0.5:
            return Node(feature_index=random.randint(0, num_features - 1))
        else:
            return Node(value=np.random.normal(0,1,1))

    operator = random.choice(operators)
    node = Node(value=operator)

    if operator in unary_operators:
        node.left = random_tree(depth - 1, num_features)
        node.right = None
    else:
        node.left = random_tree(depth - 1, num_features)
        node.right = random_tree(depth - 1, num_features)

    return node

def create_population(num_peop,depth,num_features):
    population = []
    num_ones = num_peop//2
    for i in range(num_ones):
        baby=random_tree(1,num_features)
        population.append(baby)
    for i in range(num_peop-num_ones):
        baby=random_tree(depth,num_features)
        population.append(baby)
    return population

def cost(genome,x, y):
    predictions = np.array([genome.evaluate(x[:, i]) for i in range(x.shape[1])])
    mse = np.mean((predictions - y) ** 2)
    return mse

In [84]:
meric = Node(value=np.add,
             left=Node(feature_index=0),
             right=Node(value=np.multiply,
                        left=Node(value=2),
                        right=Node(value=np.sin,
                                   left=Node(value=np.add,
                                            left=Node(feature_index=1),
                                            right=Node(value=10)))))

In [ ]:
x=[2,3]
print(meric.evaluate(x))

In [ ]:
# Collect all nodes in the tree
def collect_nodes(n, nodes):
    if n is None:
        return
    nodes.append(n)
    collect_nodes(n.left, nodes)
    collect_nodes(n.right, nodes)

def mutation(individual, feature_count):
    """
    Randomly modifies a node's value or feature_index in the tree.
    
    Args:
        node (Node): The root of the tree.
        feature_count (int): The number of input features (to determine valid feature indices).
        max_constant (float): The maximum absolute value for random constants.

    Returns:
        bool: True if a node was modified, False otherwise.
    """
    node = copy.deepcopy(individual)
    nodes = []
    collect_nodes(node, nodes)

    # Randomly pick a node
    if not nodes:
        return False

    target_node = random.choice(nodes)

    # Modify the target node
    if target_node.feature_index is not None: # Node is Xn
        if random.random() < 0.5:
        # Modify the feature index
            target_node.feature_index = random.randint(0, feature_count - 1) #TODO: find a better way to exclude existing feature index
        else:
            # Assign constant value to feature node
            target_node.feature_index = None
            target_node.value = np.random.normal(0,1,1)
    else:
    # Modify the operator or constant
        if target_node.value in operators:
            if target_node.value in unary_operators: # If the operator is one-argument, pick another one-argument operator
                target_node.value = np.random.choice([op for op in unary_operators if op != target_node.value])
            else: # If the operator is two-argument, pick another two-argument operator
                target_node.value = np.random.choice([op for op in set(operators)-set(unary_operators) if op != target_node.value])
        else: # If the node is a constant, assign a new constant value
            if random.random() < 0.5:
                # Replace the constant value with constant value
                target_node.value = np.random.normal(0,1,1)
            else:
                # Replace the constant value with a feature
                target_node.value = None
                target_node.feature_index = random.randint(0, feature_count - 1)
    return node

# Example tree
# root = Node(
#     value=np.add,
#     left=Node(value=None, feature_index=0),
#     right=Node(value=np.multiply, left=Node(value=None, feature_index=1), right=Node(value=5)),
# )
root = Node(value=np.add,
             left=Node(feature_index=0),
             right=Node(value=np.multiply,
                        left=Node(value=2),
                        right=Node(value=np.sin,
                                   left=Node(value=np.add,
                                            left=Node(feature_index=1),
                                            right=Node(value=10)))))
print("Original tree:")
print(root)

# Modify a random node
modified = mutation(root, feature_count=3)

if modified:
    print("\nModified tree:")
    print(root)
else:
    print("\nNo modification made.")


In [ ]:
my_population = create_population(10,3,3)
for i in range(len(my_population)):
    print(my_population[i])

In [ ]:
for ind in range(len(my_population)):
    if random.random() < 0.5:
        my_population[ind] = mutation(my_population[ind], 3)
        print("Mutation done")
    else:
        print("No mutation made.")

In [ ]:
for i in range(len(my_population)):
    print(my_population[i])

In [64]:
problem0 = np.load("data/problem_0.npz")
x_train = problem0["x"]
y_train = problem0["y"]
print("x")
print(x_train.shape)
print(x_train.mean(axis=1))
print(x_train.max(axis=1))
print(x_train.min(axis=1))
print(x_train.std(axis=1))
print("y")
print(y_train.shape)
print(y_train.mean(axis=0))
print(y_train.max(axis=0))
print(y_train.min(axis=0))
print(y_train.std(axis=0))

In [ ]:
problem1 = np.load("data/problem_1.npz")
x_train = problem1["x"]
y_train = problem1["y"]
print("x")
print(x_train.shape)
print(x_train.mean(axis=1))
print(x_train.max(axis=1))
print(x_train.min(axis=1))
print(x_train.std(axis=1))
print("y")
print(y_train.shape)
print(y_train.mean(axis=0))
print(y_train.max(axis=0))
print(y_train.min(axis=0))
print(y_train.std(axis=0))

In [122]:
def crossover(parent1, parent2):
    """
    Perform crossover between two parents.
    
    Args:
        parent1 (Node): First parent.
        parent2 (Node): Second parent.
        
    Returns:
        child1 (Node): First child.
        child2 (Node): Second child.
    """
    # Copy parent to create new children
    child1 = copy.deepcopy(parent1)
    child2 = copy.deepcopy(parent2)

    # Collect all nodes in the trees
    nodes1 = []
    collect_nodes(child1, nodes1)
    nodes2 = []
    collect_nodes(child2, nodes2)
    
    # Randomly pick a node from each parent
    if not nodes1 or not nodes2:
        return None
    target_node1 = random.choice(nodes1)
    target_node2 = random.choice(nodes2)

    # Copy the target node from parent1
    copy_target_node1 = copy.deepcopy(target_node1)

    # Replace the target node with the target node from parent2
    target_node1.value = target_node2.value
    target_node1.feature_index = target_node2.feature_index
    target_node1.left = target_node2.left
    target_node1.right = target_node2.right

    # Replace the target node with the target node from parent1
    target_node2.value = copy_target_node1.value
    target_node2.feature_index = copy_target_node1.feature_index
    target_node2.left = copy_target_node1.left
    target_node2.right = copy_target_node1.right
    
    return child1, child2

In [123]:
a = random_tree(2,2)
b = random_tree(2,2)

In [ ]:
print(a)
print()
print(b)

In [125]:
child1, child2 = crossover(a,b)

In [ ]:
print(a)
print()
print(b)

In [ ]:
print(child1)
print()
print(child2)

In [131]:
def simplify(population, x, y):
    """
    Simply population by removing overflowed individuals.

    Args:
        population (list): List of individuals.
        x (array-like): Input data.

    Returns:
        simplified_population (list): List of simplified individuals.
    """
    simplified_population = []
    for individual in population:
        try:
            cost(individual, x, y)
            simplified_population.append(individual)
        except:
            pass
    return simplified_population


In [ ]:
my_population = create_population(10,3,2)
for i in range(len(my_population)):
    print(my_population[i])

In [133]:
simplified_population = simplify(my_population, x_train, y_train)

In [ ]:
for i in simplified_population:
    print(i)

In [ ]:
a = [1,2,3,4,5,6,7,8,9,10]
b = [3,5,7]

# Remove elements from a whose indexes are in b
a.remove()
del a[b]

In [2]:
import numpy as np
a = np.arange(100)

In [ ]:
a

In [4]:
a = a*10

In [ ]:
a

In [ ]:
import numpy as np

def select_k_winners(arr, n, k, sorted=False):
    """
    Select k winners from the array by repeatedly comparing random n elements.
    
    Parameters:
    - arr (np.ndarray): The input array.
    - n (int): Number of elements to randomly select for each comparison.
    - k (int): Number of winners to select.
    
    Returns:
    - np.ndarray: Array of the k largest winners.
    """
    
    # Convert to a mutable list of indices
    indices = np.arange(len(arr))
    
    while len(indices) > k:
        # Randomly select `n` indices
        selected_indices = np.random.choice(indices, n, replace=False)
        
        # Find the index of the maximum among the selected
        max_idx = selected_indices[np.argmax(arr[selected_indices])]
        
        # Remove all selected indices except the winner
        indices = np.setdiff1d(indices, selected_indices)
        indices = np.append(indices, max_idx)
    
    # Extract and sort the k winners
    winners = arr[indices]
    
    if sorted:
        return np.sort(winners[:k])[::-1]
    return winners

# Example Usage
arr = np.arange(0, 1000, 10)
n = 3  # Number of elements to compare
k = 5  # Number of winners
winners = select_k_winners(arr, n, k)
print("Top K Winners:", winners)


In [ ]:
root = Node(value=np.add,
             left=Node(feature_index=0),
             right=Node(value=np.multiply,
                        left=Node(value=2),
                        right=Node(value=np.sin,
                                   left=Node(value=np.add,
                                            left=Node(feature_index=1),
                                            right=Node(value=10)))))

In [1]:
import numpy as np
from src.utils import *
from tqdm import tqdm

In [2]:
my_node1 = Node(value=0.2)
my_node2 = Node(value=0.3)
my_node3 = Node(value=0.1)
my_node4 = Node(value=0.6)
my_pop = [Individual(my_node4, age=4),Individual(my_node4, age=2),Individual(my_node4, age=1),Individual(my_node4, age=7),Individual(my_node2),Individual(my_node2),Individual(my_node2),Individual(my_node2),Individual(my_node1), Individual(my_node1),Individual(my_node1),Individual(my_node1),Individual(my_node3),Individual(my_node3),Individual(my_node3),Individual(my_node3) ]

In [3]:
abs_node = Node(value=np.abs,
                left=Node(value=np.abs,
                          left=Node(value=np.sin,
                                    left=Node(value=0.3))))
abs_ind = Individual(abs_node)
print(abs_ind)

Individual(genome=abs(abs(sin(0.3))), fitness=None, age=0, T=1)


In [3]:
# simplify(my_inidvidual[0].genome)
problem_0 = np.load("data/problem_4.npz")
x = problem_0["x"]
y = problem_0["y"]
print("x.shape:", x.shape)
print("y.shape:", y.shape)

x.shape: (2, 5000)
y.shape: (5000,)


In [5]:
simplify_operation(abs_ind.genome)
abs_ind

Individual(genome=abs(sin(0.3)), fitness=None, age=0, T=1)

In [7]:
print(abs_ind.genome.left.value)
print(type(abs_ind.genome.left.value))

<ufunc 'sin'>
<class 'numpy.ufunc'>


In [4]:
# my_pop=[abs_ind]
assign_population_fitness(my_pop,x,y)
for ind in range(len(my_pop)):
    print(ind, my_pop[ind])

0 Individual(genome=0.6, fitness=np.float64(23.581402809182254), age=4, T=1)
1 Individual(genome=0.6, fitness=np.float64(23.581402809182254), age=2, T=1)
2 Individual(genome=0.6, fitness=np.float64(23.581402809182254), age=1, T=1)
3 Individual(genome=0.6, fitness=np.float64(23.581402809182254), age=7, T=1)
4 Individual(genome=0.3, fitness=np.float64(24.51127448521074), age=0, T=1)
5 Individual(genome=0.3, fitness=np.float64(24.51127448521074), age=0, T=1)
6 Individual(genome=0.3, fitness=np.float64(24.51127448521074), age=0, T=1)
7 Individual(genome=0.3, fitness=np.float64(24.51127448521074), age=0, T=1)
8 Individual(genome=0.2, fitness=np.float64(24.861231710553575), age=0, T=1)
9 Individual(genome=0.2, fitness=np.float64(24.861231710553575), age=0, T=1)
10 Individual(genome=0.2, fitness=np.float64(24.861231710553575), age=0, T=1)
11 Individual(genome=0.2, fitness=np.float64(24.861231710553575), age=0, T=1)
12 Individual(genome=0.1, fitness=np.float64(25.231188935896405), age=0, T=1)


In [6]:
my_pop = deduplicate_population(my_pop)
for ind in range(len(my_pop)):
    print(ind, my_pop[ind])

0 Individual(genome=0.6, fitness=np.float64(23.581402809182254), age=1, T=1)
1 Individual(genome=0.3, fitness=np.float64(24.51127448521074), age=0, T=1)
2 Individual(genome=0.2, fitness=np.float64(24.861231710553575), age=0, T=1)
3 Individual(genome=0.1, fitness=np.float64(25.231188935896405), age=0, T=1)


In [ ]:
tournament_selection(my_pop,3,4,ELITISM=False)

In [ ]:
val=np.array([32])
if np.abs:
    print("Hey")

In [ ]:
a = 0.2313212132
b = 0.2313212132
print(f"Fitness: {a:.2f}, Genome: {b}")

In [7]:
my_node = Node(value=np.cos,
               left=Node(feature_index=0))
my_inidvidual = [Individual(my_node)]

In [ ]:
print(my_inidvidual[0])
print(my_inidvidual[0].genome.complexity)

In [9]:
kill_constant(my_inidvidual)

In [ ]:
my_inidvidual

In [11]:
my_node = Node(value=np.array([0.1]))
my_inidvidual = [Individual(my_node)]

In [ ]:
print(my_inidvidual[0])
print(my_inidvidual[0].genome.complexity)

In [13]:
kill_constant(my_inidvidual)

In [ ]:
my_inidvidual

In [36]:
a = list(range(20))

In [37]:
a[:] = [i for i in a if not(i % 3 == 0 and i % 2 == 0)]

In [ ]:
a

In [ ]:
for el in a:
    print(el)
    if el % 3 == 0:
        a.remove(el)
print(a)

In [1]:
import threading

In [2]:
def t1():
    for i in range(10):
        print("Thread 1")
def t2():
    for i in range(10):
        print("Thread 2")
        

In [ ]:
thread_1 = threading.Thread(target=t1)
thread_2 = threading.Thread(target=t2)

thread_2.start()
thread_1.start()
print("Done")
thread_1.join()
thread_2.join()


